https://www.kaggle.com/code/e0xextazy/basic-feature-preprocessing-stacking-upgraded

https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers

https://www.kaggle.com/code/mathchi/churn-problem-for-bank-customer

Обновление датасетов

#### Импорт либ

In [2]:
!pip install catboost lightgbm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.4 MB/s eta 0:00:00


In [3]:
from google.colab import auth

auth.authenticate_user()  # must authenticate


'''list all ids of files directly under folder folder_id'''

def folder_list(folder_id):

  from googleapiclient.discovery import build

  gdrive = build('drive', 'v3').files()

  res = gdrive.list(q="'%s' in parents" % folder_id).execute()

  return [f['id'] for f in res['files']]



'''download all files from a gdrive folder to current directory'''

def folder_download(folder_id):

  for fid in folder_list(folder_id):
    !gdown -q --id $fid

#link='https://drive.google.com/drive/folders/1RLRFrsn4vZPFIf3qmA0nCKv9txq770fZ'

folder_id="1RLRFrsn4vZPFIf3qmA0nCKv9txq770fZ"

folder_download(folder_id)
!gunzip hackaton2023_train.csv.gz
!gunzip hackaton2023_test.csv.gz

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1Is9nDmsC0h6AArYg_rdMUsmwCfo4J8Nzx1Bcds4EKOo 

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

from catboost import CatBoostClassifier, CatBoostRegressor
from matplotlib import pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import KFold, cross_val_score
from sklearn.datasets import load_breast_cancer
from sklearn.impute import KNNImputer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix

from lightgbm import LGBMClassifier

from tqdm import tqdm
import itertools

from pprint import pprint
from warnings import filterwarnings

import seaborn as sns



filterwarnings("ignore", category=FutureWarning)

#### Включим логирование через MLflow

In [3]:
!pip install pyngrok --quiet
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.8/731.8 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss, f1_score, roc_auc_score, mean_squared_error


import mlflow
mlflow.set_tracking_uri('http://79.137.194.156:5000/')
import time
import os

#### Загрузка данных

In [ ]:
def read_train(dataset_name):
  df_train = pd.read_csv(f'train_{dataset_name}.csv', sep=',')

  target = df_train['date_diff_post']
  df_train = df_train.drop('date_diff_post', axis=1)


  return df_train, target




###### import libs

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
#from sklearn.ensemble import VotingClassifier


from xgboost import XGBClassifier
from catboost import CatBoostClassifier



###### Preprocessing pipeline

Надо дописать раздельнный препроцессинг для регрессии и классификации

##### Create a stacking classifier

In [ ]:
model_metrics = []
from functools import partial

def get_metrics(y_true, y_pred, y_proba):
  SC_RMSE = mean_squared_error(y_true, y_pred)

  ndf = [SC_RMSE]
  columns = ['RMSE']
  return dict(zip(columns, ndf))

def prepare_submit(trained_model, X_test):
  y_test = trained_model.predict(X_test)
  X_submit = pd.DataFrame(X_test['customer_id'])
  X_submit['date_post_diff'] = y_test
  return X_submit

In [ ]:

from sklearn.utils.class_weight import compute_class_weight

#TRAIN_DATASET_NAME = 'train_user_order_aggs_v2.csv'
#TEST_DATASET_NAME = 'test_user_order_aggs_v2.csv'
datasets = ''.split()
df_test = pd.read_csv(f'test_user_order_aggs_v2.csv', sep=',')
for i, dataset_name in enumerate(datasets):
  #dataset_name = 'user_order_aggs_v3'

  df_train, target = read_train(dataset_name)
  #dataset_name = 'user_order_aggs'
  model_name = 'CatBoost_colab_regression_002'
  classes = np.unique(target)
  weights = compute_class_weight(class_weight='balanced', classes=classes, y=target)

  folds = 4

  # Train-Test Separation
  X_train, X_val, y_train, y_val = train_test_split(df_train, target, test_size=(1/folds), random_state=12345)
  model = CatBoostRegressor(random_state=13, task_type='CPU', class_weights=weights)
  grid = {'learning_rate': [0.03, 0.1, 0.5, 0.01],
          'depth': [4, 6, 10],
          'l2_leaf_reg': [1, 3, 5, 7, 9],
          'n_estimators':[100, 1000,4000]}
  #grid = {'learning_rate': [0.03], 'n_estimators':[10]}
  randomized_search_result = model.randomized_search(grid,
                                                    X=df_train,
                                                    y=target,
                                                    n_iter=10,
                                                    plot=True)
  best_params = randomized_search_result['params']
  print(best_params)
  trained_model = CatBoostRegressor(random_state=13, class_weights=weights, **best_params, task_type='CPU').fit(df_train, target)
  X = X_train
  with mlflow.start_run(run_name=f'grafML_{dataset_name}_{model_name}_Train'):
    y_pred = trained_model.predict(X)
    y_proba = trained_model.predict_proba(X)[:,1]
    metrics_dict = get_metrics(y_train, y_pred, y_proba)

    mlflow.log_metrics(metrics_dict)
    mlflow.log_param('catboost', str(best_params))

  X = X_val
  with mlflow.start_run(run_name=f'grafML_{dataset_name}_{model_name}_Validation'):
    y_pred = trained_model.predict(X)
    y_proba = trained_model.predict_proba(X)[:,1]
    metrics_dict = get_metrics(y_val, y_pred, y_proba)

    mlflow.log_metrics(metrics_dict)
    mlflow.log_param('catboost', str(best_params))




  X_submit = prepare_submit(trained_model, df_test)
  X_submit.to_csv(f'{dataset_name}_{model_name}_submit.csv', index=False)


##### Metrics

Надо еще f1